In [1]:
import pandas as pd
import re
import numpy as np
import nltk


df = pd.read_csv('./gender.csv',index_col=0,low_memory=False)
# df = df[~df['response_time'].isna()].copy()

def preprocess_text(text):
    if isinstance(text,float)!=True:
        # replace @... to None
        text = text.lower()
        pattern = r'@\w+'
        text = re.sub(pattern, '', text)
        text = re.sub('[^a-zA-Z]+', ' ', text)
        # text = re.sub('united','',text)
        return text.strip()
    else:
        return np.nan


from nltk.sentiment import SentimentIntensityAnalyzer
df['text'] = df['text'].apply(lambda x: preprocess_text(x))

In [2]:
df.head()

,Unnamed: 0,id,public_metrics,conversation_id,reply_settings,referenced_tweets,text,edit_history_tweet_ids,created_at,author_id,...,duration_minutes,length,similarity,similarity_2,first_name,last_name,race,gender_description,gender_name,gender
0,0,1641574347847651328,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1641574347847651328,everyone,NaN,another fail by almost hrs delayed for caterin...,['1641574347847651328'],2023-03-30 22:53:18+00:00,1538911384590643200,...,16.283333,14,0.780649,0.629673,scott,grainger,white,NaN,male,male
1,1,1641565209759473666,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1641565209759473666,everyone,NaN,a big thank you to the crew on united flight u...,['1641565209759473666'],2023-03-30 22:16:59+00:00,832282088,...,43.116667,10,0.690359,0.687843,krisy,mean,white,NaN,unknown,unknown
2,2,1641533918829588481,"{'retweet_count': 0, 'reply_count': 4, 'like_c...",1641493451198935068,everyone,"[{'type': 'replied_to', 'id': '164149345119893...",how many hours from newark,['1641533918829588481'],2023-03-30 20:12:39+00:00,28451842,...,40.266667,6,0.823931,0.610950,nicole,NaN,unknown,NaN,female,female
3,3,1641527058579836928,"{'retweet_count': 2, 'reply_count': 1, 'like_c...",1641527058579836928,everyone,NaN,what a great perk getting into the iad polaris...,['1641527058579836928'],2023-03-30 19:45:23+00:00,40876316,...,2.050000,12,0.817635,0.688270,dana,freeman,white,NaN,female,female
4,4,1641523867268046853,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1641523867268046853,everyone,NaN,thank you heidi and mr lopez at dca for helpin...,['1641523867268046853'],2023-03-30 19:32:42+00:00,3044250214,...,10.533333,5,0.831798,0.656835,janet,NaN,unknown,NaN,female,female


In [5]:
from transformers import pipeline
 
# Set up the inference pipeline using a model from the 🤗 Hub
sentiment_analysis = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
 
# Let's run the sentiment analysis on each text
sentiments = []
for text in df['text']:
   try:
     sentiment = sentiment_analysis(text)
     sentiments.append(sentiment[0]['label'])
 
   except:
     pass

Downloading: 100%|██████████| 2.76k/2.76k [00:00<00:00, 1.26MB/s]
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [6]:
df['sentiment'] = sentiments
df['sentiment'].value_counts()

NEG    36504
NEU    24649
POS    14268
Name: sentiment, dtype: int64

In [7]:
df.to_csv("./sentiment.csv")